In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
home_dir = '/content/gdrive/My Drive/Colab Notebooks/'

In [0]:
import pickle

with open(home_dir + 'dataset.bin', 'rb') as out:
    dataset = pickle.load(out)

In [0]:
Y = []
X = []

for x in dataset:
    Y.append(x[0])
    X.append(x[1])

In [5]:
print(len(Y))
print(len(Y[0]))
print(len(X))
print(len(X[0]))

42255
5
42255
768


In [0]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import model_from_json
from sklearn.model_selection import train_test_split
import json
from IPython.display import clear_output

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot = np_utils.to_categorical(encoded_Y)

In [8]:
print(list(encoder.classes_))

['angry', 'happy', 'relaxed', 'sad']


In [0]:
numpy.random.seed(7)
X_data = numpy.array(X)
X_data.reshape((42255, 1, 768))

array([[[ 0.590295  ,  0.06984732,  0.03084871, ...,  0.23810041,
          0.2490385 ,  0.33005044]],

       [[ 0.41395926,  0.10939281, -0.25555792, ...,  0.6654529 ,
          0.30756867, -0.01532822]],

       [[ 0.10382067, -0.00712498,  0.10896952, ...,  0.4952538 ,
          0.8795851 , -0.00121327]],

       ...,

       [[-0.17167349, -0.35625038, -0.11082935, ..., -0.06657284,
          0.1967138 ,  0.43955132]],

       [[ 0.08400286,  0.12135616,  0.15247023, ...,  0.08758762,
          0.6636649 ,  0.47544834]],

       [[ 0.19373825,  0.10527231,  0.308477  , ...,  0.19365042,
          0.32745823, -0.28359792]]], dtype=float32)

In [0]:
X_data.shape

(42255, 768)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_one_hot, test_size=0.33, random_state=7)

In [0]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(768, )))

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(4, activation='softmax'))

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
hist = model.fit(X_train, y_train,epochs=20, validation_split=0.1,  batch_size=10, verbose=1)

Train on 25479 samples, validate on 2831 samples
Epoch 1/20
25479/25479 [==============================] - 5s 183us/step - loss: 0.4106 - acc: 0.8168 - val_loss: 0.4173 - val_acc: 0.8123
Epoch 2/20
25479/25479 [==============================] - 4s 143us/step - loss: 0.4103 - acc: 0.8166 - val_loss: 0.4136 - val_acc: 0.8124
Epoch 3/20
25479/25479 [==============================] - 4s 149us/step - loss: 0.4091 - acc: 0.8170 - val_loss: 0.4144 - val_acc: 0.8118
Epoch 4/20
25479/25479 [==============================] - 4s 155us/step - loss: 0.4080 - acc: 0.8179 - val_loss: 0.4228 - val_acc: 0.8076
Epoch 5/20
25479/25479 [==============================] - 4s 146us/step - loss: 0.4079 - acc: 0.8188 - val_loss: 0.4209 - val_acc: 0.8081
Epoch 6/20
25479/25479 [==============================] - 4s 146us/step - loss: 0.4075 - acc: 0.8188 - val_loss: 0.4235 - val_acc: 0.8076
Epoch 7/20
25479/25479 [==============================] - 4s 146us/step - loss: 0.4062 - acc: 0.8184 - val_loss: 0.4319 - v

In [0]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 79.75%


In [0]:
model_json = model.to_json()
with open(home_dir + "model_0.7975.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(home_dir + "model_0.7975.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
json_file = open(home_dir + "model_0.7975.json", "r")
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(home_dir + "model_0.7975.h5")

In [0]:
loaded_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
scores = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

acc: 79.75%


In [0]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

# Initialize the constructor
model1 = Sequential()

# Add an input layer 
model1.add(Dense(1024, activation='relu', input_shape=(768, )))

# Add one hidden layer 
model1.add(Dense(1024, activation='relu'))

# Add an output layer 
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 1024)              787456    
_________________________________________________________________
dense_60 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_61 (Dense)             (None, 4)                 4100      
Total params: 1,841,156
Trainable params: 1,841,156
Non-trainable params: 0
_________________________________________________________________


In [0]:
hist = model1.fit(X_data, y_one_hot,epochs=10, validation_split=0.2, batch_size=100, verbose=1)

Train on 33804 samples, validate on 8451 samples
Epoch 1/10
33804/33804 [==============================] - 16s 484us/step - loss: 0.4840 - acc: 0.7814 - val_loss: 0.4675 - val_acc: 0.7787
Epoch 2/10
33804/33804 [==============================] - 15s 452us/step - loss: 0.4439 - acc: 0.7967 - val_loss: 0.4397 - val_acc: 0.8030
Epoch 3/10
33804/33804 [==============================] - 16s 474us/step - loss: 0.4241 - acc: 0.8080 - val_loss: 0.4504 - val_acc: 0.7956
Epoch 4/10
33804/33804 [==============================] - 15s 453us/step - loss: 0.4060 - acc: 0.8173 - val_loss: 0.4363 - val_acc: 0.8018
Epoch 5/10
33804/33804 [==============================] - 15s 448us/step - loss: 0.3856 - acc: 0.8277 - val_loss: 0.4525 - val_acc: 0.7970
Epoch 6/10
33804/33804 [==============================] - 15s 453us/step - loss: 0.3621 - acc: 0.8394 - val_loss: 0.4656 - val_acc: 0.7985
Epoch 7/10
33804/33804 [==============================] - 15s 455us/step - loss: 0.3415 - acc: 0.8500 - val_loss: 0.4

In [0]:
save(model1, X_test, y_test)

[0.32395218722963043, 0.8692183578428978]


In [0]:
#1024 x2 0
#1024 x1 100
#128 x 2 / LOST
#12/8 200
#1024/512/256 300
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

# Initialize the constructor
model_end = Sequential()

# Add an input layer 
model_end.add(Dense(1024, activation='relu', input_shape=(768, )))

model_end.add(Dense(512, activation='relu'))

model_end.add(Dense(256, activation='relu'))

# Add an output layer 
model_end.add(Dense(4, activation='softmax'))

model_end.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
def save(model, X_test, Y_test, X_train, Y_train, cnt):
  scores = model.evaluate(X_test, Y_test, verbose=0)
  scores_train = model.evaluate(X_train, Y_train)
  test_score = scores[1]*100
  train_score = scores_train[1]*100
  print("test", test_score)
  print("train", train_score)
  name = "model_" + str(cnt) + "_" + str(train_score) + "_" + str(test_score)
  
  model_json = model.to_json()
  with open(home_dir + "models/" + name + ".json", "w") as json_file:
    json_file.write(model_json)
  model.save_weights(home_dir + "models/" + name + ".h5")
  

In [0]:
max_epochs = 200
epoch_step = 10

cnt = 300
while max_epochs > 0:
  model_end.fit(X_train, y_train,epochs=epoch_step, validation_split=0.2, batch_size=100, verbose=0)
  
  save(model_end, X_test, y_test, X_train, y_train, cnt)
  cnt += 1
  
  max_epochs-=epoch_step

28310/28310 [==============================] - 4s 135us/step
test 80.1613481483309
train 88.94736842189481
28310/28310 [==============================] - 4s 159us/step
test 79.18608820408465
train 94.26704344670287
28310/28310 [==============================] - 5s 161us/step
test 79.01398350492349
train 95.21282232468813
28310/28310 [==============================] - 4s 142us/step
test 78.50663319673534
train 94.72447898395488
28310/28310 [==============================] - 5s 172us/step
test 79.16636787635444
train 95.59078064372454
28310/28310 [==============================] - 4s 152us/step
test 79.04983865056427
train 95.50600494482794
28310/28310 [==============================] - 4s 142us/step
test 78.64288275196544
train 95.29406569906024
28310/28310 [==============================] - 4s 153us/step
test 78.49408390317694
train 95.09537265983752
28310/28310 [==============================] - 4s 139us/step
test 79.28827536922495
train 95.71882726909499
28310/28310 [================

In [0]:
def classify(encoded, model_name):
    json_file = open(model_name + ".json", "r")
    loaded_model_json = json_file.read()
    json_file.close()

    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_name + ".h5")
    
    #compile_model
    loaded_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    res = loaded_model.predict(encoded)
    
    return res

In [0]:
def make_final(res):
  length = len(res)
  summed = [sum(x) for x in zip(*res)]
  final_res = [x/length for x in summed]
  return final_res

In [0]:
# file = open(home_dir + 'classify_lyric.bin', 'rb')
# encoded = pickle.load(file)
# res = classify(encoded, home_dir + 'model_0.7975')

In [0]:
# make_final(res)

In [0]:
def load_model(model_name):
  json_file = open(model_name + ".json", "r")
  loaded_model_json = json_file.read()
  json_file.close()

  loaded_model = model_from_json(loaded_model_json)
  loaded_model.load_weights(model_name + ".h5")

  #compile_model
  loaded_model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])
  return loaded_model

In [0]:
def classify_batch(batch_name, model):
  res_objs = []
  with open(home_dir + 'encoded/' + batch_name, 'rb') as out:
    batch_data = pickle.load(out)
  length = len(batch_data)
  cnt = 1
  for x in batch_data:
    clear_output()
    print(cnt, "/", length)
    cnt+=1
    res = model.predict(x['encoded'])
    final_res = make_final(res)
    new_res_obj = {
       'id': x['id'],
       'author': x['author'],
       'song_name': x['song'],
       'lyrics': x['lyrics'],
       'angry_score': final_res[0],
       'happy_score': final_res[1],
       'relaxed_score': final_res[2],
       'sad_score': final_res[3]
    }
    res_objs.append(new_res_obj)
    new_batch_name = batch_name.replace(".bin", ".json")
    with open(home_dir + "final_data/" + new_batch_name, 'w') as file:
      json.dump(res_objs, file, indent=4)
  

In [0]:
def classify_test(arr, model_name):
  res_objs = []
  for x in arr:
    res = classify(x['encoded'], model_name)
    final_res = make_final(res)
    new_res_obj = {
        'id': x['id'],
        'author': x['author'],
        'song_name': x['song'],
        'lyrics': x['lyrics'],
        'angry_score': final_res[0],
        'happy_score': final_res[1],
        'relaxed_score': final_res[2],
        'sad_score': final_res[3]
    }
    res_objs.append(new_res_obj)
  with open(home_dir + "final_data/test.json", 'w') as file:
    json.dump(res_objs, file, indent=4)

In [0]:
# file = open(home_dir + 'classify_lyric.bin', 'rb')
# encoded = pickle.load(file)

In [0]:
# test_arr = [
#     {
#         "id": "0_0",
#         "author": "test_author",
#         "song": "test_song",
#         "lyrics": "test_lyrics",
#         "encoded": encoded
#     }
# ]

In [0]:
# test_arr

In [0]:
# classify_test(test_arr, home_dir + "model_100_87.82409042951623_80.49300824711084")

In [0]:
model = load_model(home_dir + "model_100_87.82409042951623_80.49300824711084")

In [41]:
classify_batch("1_10000_batch.bin",  model)

10000 / 10000
